In [17]:
import os
import cv2
import numpy as np
import rasterio
from glob import glob

# Define target size and format
TARGET_SIZE = (512, 512)
OUTPUT_FORMAT = '.jpeg'  # Choose JPEG or PNG
BASE_OUTPUT_PATH = "C:/Users/Pranavi/Desktop/WHALESPROJECT/processed"

# Define individual output folders
output_paths = {
    'ship': os.path.join(BASE_OUTPUT_PATH, 'ship'),
    'fin_whale': os.path.join(BASE_OUTPUT_PATH, 'fin_whale'),
    'other': os.path.join(BASE_OUTPUT_PATH, 'other')
}

# Ensure output folders exist
for path in output_paths.values():
    os.makedirs(path, exist_ok=True)

# Dictionary of input folders
input_paths = {
    'ship': r'C:\Users\Pranavi\Desktop\WHALESPROJECT\final\ship\*.png',
    'fin_whale': [
        r'C:\Users\Pranavi\Desktop\WHALESPROJECT\final\fin whale\oceansat\*.jpg',
        r'C:\Users\Pranavi\Desktop\WHALESPROJECT\final\fin whale\Pelagos2016\*.PNG',
        r'C:\Users\Pranavi\Desktop\WHALESPROJECT\final\fin whale\Thermal\*.png'
    ],
    'other': [
        r'C:\Users\Pranavi\Desktop\WHALESPROJECT\final\other\aerial\*.jpg',
        r'C:\Users\Pranavi\Desktop\WHALESPROJECT\final\other\googleEarth\*.jpg',
        r'C:\Users\Pranavi\Desktop\WHALESPROJECT\final\other\ukdatacenter\*.PNG'
    ]
}

def preprocess_image(image_path, output_folder):
    # Load image using rasterio if it's geospatial or OpenCV otherwise
    try:
        with rasterio.open(image_path) as src:
            img = src.read([1, 2, 3]) if src.count >= 3 else src.read(1)
            img = np.moveaxis(img, 0, -1)  # Channels last
    except:
        img = cv2.imread(image_path)

    # Radiometric correction (normalize pixel values)
    img = img.astype(np.float32)
    img -= img.min()
    img /= img.max()  # Scale between 0 and 1
    img *= 255
    img = img.astype(np.uint8)

    # Resize image to target size
    img_resized = cv2.resize(img, TARGET_SIZE)

    # Save in target format
    filename = os.path.splitext(os.path.basename(image_path))[0] + OUTPUT_FORMAT
    cv2.imwrite(os.path.join(output_folder, filename), img_resized)

# Process each dataset
for category, paths in input_paths.items():
    if isinstance(paths, list):  # Multiple folders in a category
        for path in paths:
            for image_path in glob(path):
                preprocess_image(image_path, output_paths[category])
    else:
        for image_path in glob(paths):
            preprocess_image(image_path, output_paths[category])

print("Datasets processed and saved successfully!")


C:\Users\Pranavi\AppData\Local\Temp\ipykernel_11760\1737393571.py:40: RuntimeWarning: invalid value encountered in divide
  img /= img.max()  # Scale between 0 and 1
C:\Users\Pranavi\AppData\Local\Temp\ipykernel_11760\1737393571.py:42: RuntimeWarning: invalid value encountered in cast
  img = img.astype(np.uint8)


Datasets processed and saved successfully!


In [18]:
import cv2
import numpy as np
import os


# Function for applying Gaussian filter
def apply_gaussian_filter(image, sigma=1):
    blurred_image = cv2.GaussianBlur(image, (0, 0), sigmaX=sigma, sigmaY=sigma)
    return blurred_image

# Function for contrast stretching
def contrast_stretching(image):
    min_val = image.min()
    max_val = image.max()
    stretched_image = (image - min_val) / (max_val - min_val) * 255
    stretched_image = stretched_image.astype('uint8')
    return stretched_image

# Function to preprocess and save images
def preprocess_and_save_images(input_folder, output_folder, sigma):
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpeg"): 
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if image is None:
                print(f"Error reading image {filename}")
                continue

            # Apply Gaussian filter
            image = apply_gaussian_filter(image, sigma)

            # Apply contrast stretching
            image = contrast_stretching(image)

            # Save the processed image
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, image)
            print(f"Processed and saved {filename}")

# Example usage
input_folder = r'C:\Users\Pranavi\Downloads\all_frames (5)\processed\other\fin_whale'
output_folder = r'C:\Users\Pranavi\Downloads\all_frames (5)\preprocessed\other'

sigma = 1.0

preprocess_and_save_images(input_folder, output_folder, sigma)


Processed and saved frame_00001.jpeg
Processed and saved frame_00002.jpeg
Processed and saved frame_00003.jpeg
Processed and saved frame_00004.jpeg
Processed and saved frame_00005.jpeg
Processed and saved frame_00006.jpeg
Processed and saved frame_00007.jpeg
Processed and saved frame_00008.jpeg


C:\Users\Pranavi\AppData\Local\Temp\ipykernel_11760\3817459823.py:15: RuntimeWarning: invalid value encountered in divide
  stretched_image = (image - min_val) / (max_val - min_val) * 255
C:\Users\Pranavi\AppData\Local\Temp\ipykernel_11760\3817459823.py:16: RuntimeWarning: invalid value encountered in cast
  stretched_image = stretched_image.astype('uint8')


Processed and saved frame_00009.jpeg
Processed and saved frame_00010.jpeg
Processed and saved frame_00011.jpeg
Processed and saved frame_00012.jpeg
Processed and saved frame_00013.jpeg
Processed and saved frame_00014.jpeg
Processed and saved frame_00015.jpeg
Processed and saved frame_00016.jpeg
Processed and saved frame_00017.jpeg
Processed and saved frame_00018.jpeg
Processed and saved frame_00019.jpeg
Processed and saved frame_00020.jpeg
Processed and saved frame_00021.jpeg
Processed and saved frame_00022.jpeg
Processed and saved frame_00023.jpeg
Processed and saved frame_00024.jpeg
Processed and saved frame_00025.jpeg
Processed and saved frame_00026.jpeg
Processed and saved frame_00027.jpeg
Processed and saved frame_00028.jpeg
Processed and saved frame_00029.jpeg
Processed and saved frame_00030.jpeg
Processed and saved frame_00031.jpeg
Processed and saved frame_00032.jpeg
Processed and saved frame_00033.jpeg
Processed and saved frame_00034.jpeg
Processed and saved frame_00035.jpeg
P

In [19]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.encoder = nn.Sequential(
            DoubleConv(in_channels, 64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            DoubleConv(64, 128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            DoubleConv(128, 256),
            nn.MaxPool2d(kernel_size=2, stride=2),
            DoubleConv(256, 512),
            nn.MaxPool2d(kernel_size=2, stride=2),
            DoubleConv(512, 1024),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2),
            DoubleConv(1024, 512),
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            DoubleConv(512, 256),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            DoubleConv(256, 128),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            DoubleConv(128, 64),
            nn.Conv2d(64, 3, kernel_size=1)  # Change out_channels to 3
        )

    def forward(self, x):
        enc_outputs = []
        for module in self.encoder:
            x = module(x)
            enc_outputs.append(x.clone())  # Clone the tensor to avoid in-place modification
        
        enc_outputs.pop()  # Remove the last output since it won't be used in the decoder

        for idx, module in enumerate(self.decoder):
            if isinstance(module, nn.ConvTranspose2d):
                x = module(x)
                # Resize the encoder output to match the size of the decoder output
                enc_out_resize = F.interpolate(enc_outputs[-idx - 1], size=x.shape[2:], mode='bilinear', align_corners=True)
                # Concatenate with corresponding encoder output
                x = torch.cat([x, enc_out_resize], dim=1)
            else:
                x = module(x)

        return x
# Load segmentation model state dictionary
segmentation_model_path = r"C:\Users\Pranavi\Desktop\mini2\segmentation\mini2_unet_model.pth"
segmentation_state_dict = torch.load(segmentation_model_path, map_location=torch.device('cpu'))

# Instantiate your segmentation model
segmentation_model = UNet(in_channels=3, out_channels=1)  

# Load the state dictionary into the model
segmentation_model.load_state_dict(segmentation_state_dict)
segmentation_model.eval()
# Directory containing frames
frames_directory = r'C:\Users\Pranavi\Downloads\all_frames (5)\preprocessed\other'

# Directory to save segmented images
segmented_directory = r"C:\Users\Pranavi\Downloads\all_frames (5)\segmented\other"

# Ensure the segmented directory exists, if not, create it
if not os.path.exists(segmented_directory):
    os.makedirs(segmented_directory)

# Loop through each image file in the directory
for filename in os.listdir(frames_directory):
    if filename.endswith(".jpeg"):
        # Read the image
        image_path = os.path.join(frames_directory, filename)
        image = cv2.imread(image_path)

        # Check if the image is loaded successfully
        if image is None:
            print("Error: Unable to load the image. Skipping:", image_path)
            continue

        # Preprocess the image for segmentation
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image to RGB format
        original_image = image.copy()  # Make a copy of the original image
        image = cv2.resize(image, (256, 256))  # Resize image if needed
        image_tensor = torch.from_numpy(image.transpose((2, 0, 1))).float()  # Convert to tensor and adjust dimensions
        image_tensor /= 255.0  # Normalize image

        # Perform segmentation
        with torch.no_grad():
            segmentation_result = segmentation_model(image_tensor.unsqueeze(0))

        # Postprocess the segmentation result
        thresholded_output = (segmentation_result > 0.5).float()  # Assuming output values are between 0 and 1
        segmentation_mask = thresholded_output.squeeze().cpu().numpy()  # Convert tensor to numpy array
        segmentation_mask_single_channel = segmentation_mask[0]  # Selecting the first channel as an example

        # Resize segmentation mask to match original image size
        segmentation_mask_single_channel = cv2.resize(segmentation_mask_single_channel, (original_image.shape[1], original_image.shape[0]))

        # Overlay segmentation mask on original image
        alpha = 0.5  # Blend factor
        overlay = original_image.copy()
        overlay[segmentation_mask_single_channel > 0] = [255, 0, 0]  # Red color where mask is present
        result = cv2.addWeighted(overlay, alpha, original_image, 1 - alpha, 0)

        # Save the segmented image
        segmented_image_path = os.path.join(segmented_directory, filename)
        cv2.imwrite(segmented_image_path, cv2.cvtColor(result, cv2.COLOR_RGB2BGR))

        print("Segmentation completed for:", filename)

print("All images segmented and saved successfully.")

Segmentation completed for: frame_00001.jpeg
Segmentation completed for: frame_00002.jpeg
Segmentation completed for: frame_00003.jpeg
Segmentation completed for: frame_00004.jpeg
Segmentation completed for: frame_00005.jpeg
Segmentation completed for: frame_00006.jpeg
Segmentation completed for: frame_00007.jpeg
Segmentation completed for: frame_00008.jpeg
Segmentation completed for: frame_00009.jpeg
Segmentation completed for: frame_00010.jpeg
Segmentation completed for: frame_00011.jpeg
Segmentation completed for: frame_00012.jpeg
Segmentation completed for: frame_00013.jpeg
Segmentation completed for: frame_00014.jpeg
Segmentation completed for: frame_00015.jpeg
Segmentation completed for: frame_00016.jpeg
Segmentation completed for: frame_00017.jpeg
Segmentation completed for: frame_00018.jpeg
Segmentation completed for: frame_00019.jpeg
Segmentation completed for: frame_00020.jpeg
Segmentation completed for: frame_00021.jpeg
Segmentation completed for: frame_00022.jpeg
Segmentati

In [14]:
import os

# Directory containing segmented images
segmented_images_directory = r"C:\Users\Pranavi\Downloads\fin"

# Directory to save detected images
detected_images_directory = r"C:\Users\Pranavi\Desktop\mini2\classification\dataset\fin_whale"
os.makedirs(detected_images_directory, exist_ok=True)

# Loop through each segmented image file in the directory
for filename in os.listdir(segmented_images_directory):
    if filename.endswith(".jpg"):
        # Path to the segmented image
        segmented_image_path = os.path.join(segmented_images_directory, filename)

        # Perform object detection using YOLO on the segmented image
        !yolo task=detect mode=predict model=r'C:\Users\Pranavi\Desktop\mini2\detection\best.pt' conf=0.25 source={segmented_image_path} save=True

print("Object detection completed for all segmented images.")


Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs

image 1/1 C:\Users\Pranavi\Downloads\fin\frame_0.jpg: 480x800 1 fin whale, 182.9ms
Speed: 9.5ms preprocess, 182.9ms inference, 11.5ms postprocess per image at shape (1, 3, 480, 800)
Results saved to runs\detect\predict
 Learn more at https://docs.ultralytics.com/modes/predict
Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs

image 1/1 C:\Users\Pranavi\Downloads\fin\frame_1170.jpg: 480x800 1 fin whale, 171.4ms
Speed: 3.0ms preprocess, 171.4ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 800)
Results saved to runs\detect\predict2
 Learn more at https://docs.ultralytics.com/modes/predict
Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i5-1

Object detection completed for all segmented images.


Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs

image 1/1 C:\Users\Pranavi\Downloads\fin\frame_996.jpg: 480x800 1 fin whale, 157.7ms
Speed: 6.0ms preprocess, 157.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 800)
Results saved to runs\detect\predict1177
 Learn more at https://docs.ultralytics.com/modes/predict


In [16]:
import shutil
import os
import glob

# Base directory containing subfolders predict, predict2, etc.
base_directory = r"C:\Users\Pranavi\Desktop\WHALESPROJECT\runs\detect"

# Destination directory
destination_dir = r"C:\Users\Pranavi\Desktop\mini2\classification\dataset\fin_whale"
os.makedirs(destination_dir, exist_ok=True)

# Find all JPEG files in folders starting with "predict" within base_directory
image_paths = glob.glob(os.path.join(base_directory, "predict*", "*.jpg"))

# Initialize a counter to create unique names
counter = 1

# Copy each image to the destination directory with a new name
for image_path in image_paths:
    # Get the original extension (should be .jpeg)
    _, extension = os.path.splitext(image_path)
    
    # Create a new file name using the counter
    new_name = f"im_{counter}{extension}"
    new_path = os.path.join(destination_dir, new_name)
    
    # Copy and rename the file
    shutil.copy(image_path, new_path)
    print(f"Copied {image_path} to {new_path}")
    
    # Increment the counter
    counter += 1

print("All images copied and renamed successfully.")


Copied C:\Users\Pranavi\Desktop\WHALESPROJECT\runs\detect\predict\frame_0.jpg to C:\Users\Pranavi\Desktop\mini2\classification\dataset\fin_whale\im_1.jpg
Copied C:\Users\Pranavi\Desktop\WHALESPROJECT\runs\detect\predict10\frame_1206.jpg to C:\Users\Pranavi\Desktop\mini2\classification\dataset\fin_whale\im_2.jpg
Copied C:\Users\Pranavi\Desktop\WHALESPROJECT\runs\detect\predict100\frame_1698.jpg to C:\Users\Pranavi\Desktop\mini2\classification\dataset\fin_whale\im_3.jpg
Copied C:\Users\Pranavi\Desktop\WHALESPROJECT\runs\detect\predict1000\frame_6558.jpg to C:\Users\Pranavi\Desktop\mini2\classification\dataset\fin_whale\im_4.jpg
Copied C:\Users\Pranavi\Desktop\WHALESPROJECT\runs\detect\predict1001\frame_6564.jpg to C:\Users\Pranavi\Desktop\mini2\classification\dataset\fin_whale\im_5.jpg
Copied C:\Users\Pranavi\Desktop\WHALESPROJECT\runs\detect\predict1002\frame_6570.jpg to C:\Users\Pranavi\Desktop\mini2\classification\dataset\fin_whale\im_6.jpg
Copied C:\Users\Pranavi\Desktop\WHALESPROJEC

In [1]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image, ImageDraw, ImageFont
import torchvision.models as models
import torch.nn as nn

# Counter for non-fin whale images (for any specific logic you want to apply)
non_fin_whale_count = 0

# Define the model architecture
model = models.resnet101(pretrained=False)
num_classes = 3  # Set the number of output classes to 3
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Load the model state dictionary
model_path = r"C:\Users\Pranavi\Desktop\mini2\classification\fin_whale_classification_resnet101_model.pth"
model_state_dict = torch.load(model_path, map_location=torch.device('cpu'))

# Load the state dictionary into the model
model.load_state_dict(model_state_dict)

# Set model to evaluation mode
model.eval()

# Define the class labels
class_labels = ["fin whale", "ship", "other"]  # Updated with three classes

# Define transformations
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model's expected input size
    transforms.ToTensor(),           # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Directory containing the images
image_dir = r'C:\Users\Pranavi\Desktop\mini2\classification\mixed'

# Output directory for classified images
output_dir = r'C:\Users\Pranavi\Desktop\mini2\classification\classified'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Font settings (large size for visibility)
try:
    font = ImageFont.truetype("arial.ttf", 36)  # Increased font size to 36
except IOError:
    font = ImageFont.load_default()

# Iterate over each image in the directory
for image_file in os.listdir(image_dir):
    if image_file.endswith('.jpg') or image_file.endswith('.png') or image_file.endswith('.jpeg'):  # Filter images
        # Load the image
        image_path = os.path.join(image_dir, image_file)
        image = Image.open(image_path).convert("RGB")

        # Preprocess the image
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

        # Perform inference
        with torch.no_grad():
            output = model(input_batch)

        # Interpret the output
        predicted_class = torch.argmax(output).item()
        predicted_label = class_labels[predicted_class]

        # Draw the label on the image
        draw = ImageDraw.Draw(image)
        text_position = (20, 20)  # Position for the label on the image
        text_color = (255, 0, 0)  # Red color for visibility
        draw.text(text_position, predicted_label, fill=text_color, font=font)

        # Save the classified image
        output_subdir = os.path.join(output_dir, predicted_label)
        os.makedirs(output_subdir, exist_ok=True)
        output_path = os.path.join(output_subdir, image_file)
        image.save(output_path)

        # Increment counter if classified as "other" or "ship"
        if predicted_label == "other" or predicted_label == "ship":
            non_fin_whale_count += 1

        print(f"Saved {image_file} as {predicted_label} in {output_subdir}")

print(f"Total non-fin whale images classified: {non_fin_whale_count}")


C:\Users\Pranavi\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Pranavi\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Saved aerial_1.jpeg as other in C:\Users\Pranavi\Desktop\mini2\classification\classified\other
Saved aerial_10.jpeg as other in C:\Users\Pranavi\Desktop\mini2\classification\classified\other
Saved aerial_11.jpeg as other in C:\Users\Pranavi\Desktop\mini2\classification\classified\other
Saved aerial_12.jpeg as other in C:\Users\Pranavi\Desktop\mini2\classification\classified\other
Saved aerial_13.jpeg as other in C:\Users\Pranavi\Desktop\mini2\classification\classified\other
Saved aerial_14.jpeg as other in C:\Users\Pranavi\Desktop\mini2\classification\classified\other
Saved aerial_15.jpeg as other in C:\Users\Pranavi\Desktop\mini2\classification\classified\other
Saved aerial_16.jpeg as other in C:\Users\Pranavi\Desktop\mini2\classification\classified\other
Saved aerial_17.jpeg as other in C:\Users\Pranavi\Desktop\mini2\classification\classified\other
Saved aerial_18.jpeg as other in C:\Users\Pranavi\Desktop\mini2\classification\classified\other
Saved aerial_19.jpeg as other in C:\Users

In [19]:
import os
import cv2
import numpy as np
import rasterio

# Define target size and format
TARGET_SIZE = (512, 512)
OUTPUT_FORMAT = '.jpeg'  # Choose JPEG or PNG
BASE_OUTPUT_PATH = "C:/Users/Pranavi/Desktop/mini2/Test/processed"

# Define output folder
output_folder = BASE_OUTPUT_PATH

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Input image path
image_path = "C:/Users/Pranavi/Desktop/mini2/Test/final/Test6.png"

def preprocess_image(image_path, output_folder):
    # Load image using rasterio if it's geospatial or OpenCV otherwise
    try:
        with rasterio.open(image_path) as src:
            img = src.read([1, 2, 3]) if src.count >= 3 else src.read(1)
            img = np.moveaxis(img, 0, -1)  # Channels last
    except:
        img = cv2.imread(image_path)

    # Radiometric correction (normalize pixel values)
    img = img.astype(np.float32)
    img -= img.min()
    img /= img.max()  # Scale between 0 and 1
    img *= 255
    img = img.astype(np.uint8)

    # Resize image to target size
    img_resized = cv2.resize(img, TARGET_SIZE)

    # Save in target format
    filename = os.path.splitext(os.path.basename(image_path))[0] + OUTPUT_FORMAT
    cv2.imwrite(os.path.join(output_folder, filename), img_resized)

# Process the single image
preprocess_image(image_path, output_folder)

print("Image processed and saved successfully!")


Image processed and saved successfully!


In [21]:
import cv2
import numpy as np
import os

# Function for applying Gaussian filter
def apply_gaussian_filter(image, sigma=1):
    blurred_image = cv2.GaussianBlur(image, (0, 0), sigmaX=sigma, sigmaY=sigma)
    return blurred_image

# Function for contrast stretching
def contrast_stretching(image):
    min_val = image.min()
    max_val = image.max()
    stretched_image = (image - min_val) / (max_val - min_val) * 255
    stretched_image = stretched_image.astype('uint8')
    return stretched_image

# Function to preprocess and save a single image
def preprocess_and_save_single_image(image_path, output_folder, sigma):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if image is None:
        print(f"Error reading image {image_path}")
        return

    # Apply Gaussian filter
    image = apply_gaussian_filter(image, sigma)

    # Apply contrast stretching
    image = contrast_stretching(image)

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Save the processed image
    filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, image)
    print(f"Processed and saved {filename}")

# Example usage
input_image = r'C:\Users\Pranavi\Desktop\mini2\Test\processed\Test6.jpeg'
output_folder = r'C:\Users\Pranavi\Desktop\mini2\Test\preprocessed'

sigma = 1.0

preprocess_and_save_single_image(input_image, output_folder, sigma)


Processed and saved Test6.jpeg


In [23]:
import cv2
import os
import torch
import torch.nn.functional as F

# Load the UNet model (same as your previous code)
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.encoder = nn.Sequential(
            DoubleConv(in_channels, 64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            DoubleConv(64, 128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            DoubleConv(128, 256),
            nn.MaxPool2d(kernel_size=2, stride=2),
            DoubleConv(256, 512),
            nn.MaxPool2d(kernel_size=2, stride=2),
            DoubleConv(512, 1024),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2),
            DoubleConv(1024, 512),
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            DoubleConv(512, 256),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            DoubleConv(256, 128),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            DoubleConv(128, 64),
            nn.Conv2d(64, 3, kernel_size=1)  # Change out_channels to 3
        )

    def forward(self, x):
        enc_outputs = []
        for module in self.encoder:
            x = module(x)
            enc_outputs.append(x.clone())  # Clone the tensor to avoid in-place modification
        
        enc_outputs.pop()  # Remove the last output since it won't be used in the decoder

        for idx, module in enumerate(self.decoder):
            if isinstance(module, nn.ConvTranspose2d):
                x = module(x)
                # Resize the encoder output to match the size of the decoder output
                enc_out_resize = F.interpolate(enc_outputs[-idx - 1], size=x.shape[2:], mode='bilinear', align_corners=True)
                # Concatenate with corresponding encoder output
                x = torch.cat([x, enc_out_resize], dim=1)
            else:
                x = module(x)

        return x

# Load segmentation model state dictionary
segmentation_model_path = r"C:\Users\Pranavi\Desktop\mini2\segmentation\mini2_unet_model.pth"
segmentation_state_dict = torch.load(segmentation_model_path, map_location=torch.device('cpu'))

# Instantiate your segmentation model
segmentation_model = UNet(in_channels=3, out_channels=1)

# Load the state dictionary into the model
segmentation_model.load_state_dict(segmentation_state_dict)
segmentation_model.eval()

# Path of the image you want to segment
image_path = r"C:\Users\Pranavi\Desktop\mini2\Test\processed\Test6.jpeg"

# Directory to save the segmented image
segmented_directory = r"C:\Users\Pranavi\Desktop\mini2\Test\segmented"

# Ensure the segmented directory exists, if not, create it
if not os.path.exists(segmented_directory):
    os.makedirs(segmented_directory)

# Read the image
image = cv2.imread(image_path)

# Check if the image is loaded successfully
if image is None:
    print("Error: Unable to load the image. Skipping:", image_path)

# Preprocess the image for segmentation
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image to RGB format
original_image = image.copy()  # Make a copy of the original image
image = cv2.resize(image, (256, 256))  # Resize image if needed
image_tensor = torch.from_numpy(image.transpose((2, 0, 1))).float()  # Convert to tensor and adjust dimensions
image_tensor /= 255.0  # Normalize image

# Perform segmentation
with torch.no_grad():
    segmentation_result = segmentation_model(image_tensor.unsqueeze(0))

# Postprocess the segmentation result
thresholded_output = (segmentation_result > 0.5).float()  # Assuming output values are between 0 and 1
segmentation_mask = thresholded_output.squeeze().cpu().numpy()  # Convert tensor to numpy array
segmentation_mask_single_channel = segmentation_mask[0]  # Selecting the first channel as an example

# Resize segmentation mask to match original image size
segmentation_mask_single_channel = cv2.resize(segmentation_mask_single_channel, (original_image.shape[1], original_image.shape[0]))

# Overlay segmentation mask on original image
alpha = 0.5  # Blend factor
overlay = original_image.copy()
overlay[segmentation_mask_single_channel > 0] = [255, 0, 0]  # Red color where mask is present
result = cv2.addWeighted(overlay, alpha, original_image, 1 - alpha, 0)

# Save the segmented image
segmented_image_path = os.path.join(segmented_directory, "Test6.jpeg")
cv2.imwrite(segmented_image_path, cv2.cvtColor(result, cv2.COLOR_RGB2BGR))

print("Segmentation completed for Test1.jpeg")


Segmentation completed for Test1.jpeg


In [25]:
import os

# Path to the segmented image
segmented_image_path = r"C:\Users\Pranavi\Desktop\mini2\Test\segmented\Test6.jpeg"

# Directory to save detected images
detected_images_directory = r"C:\Users\Pranavi\Desktop\mini2\Test\detected"
os.makedirs(detected_images_directory, exist_ok=True)

# Perform object detection using YOLO on the segmented image
!yolo task=detect mode=predict model=r'C:\Users\Pranavi\Desktop\mini2\detection\best.pt' conf=0.25 source={segmented_image_path} save=True project={detected_images_directory} name=detected_images

print("Object detection completed for the segmented image.")


Object detection completed for the segmented image.


Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs

image 1/1 C:\Users\Pranavi\Desktop\mini2\Test\segmented\Test6.jpeg: 800x800 3 ships, 238.6ms
Speed: 5.5ms preprocess, 238.6ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)
Results saved to C:\Users\Pranavi\Desktop\mini2\Test\detected\detected_images
 Learn more at https://docs.ultralytics.com/modes/predict
